In [3]:
from google.colab import drive

drive.mount('/content/drive')

!ls 'drive/My Drive'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 bpm.behnejadi.rar
'Colab Notebooks'
'Copy of data.aug.rar'
 data.aug
 data.aug.rar
 data.mobilenet
 data.resnet50
 data.row
 data.row.battah
 dataset
 data.v
 data.vgg16
 data.xception
 IEEE_Iran_Section_Certificate_AIDLinAutoRobotics__Fateme_behnejadi__125598.pdf
 thesis


In [4]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from keras.optimizers import SGD
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
%matplotlib inline
from keras.wrappers.scikit_learn import KerasClassifier
import sklearn
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score
from scipy import interp
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import cross_val_score

In [5]:
AUTO = tf.data.experimental.AUTOTUNE
# Cluster Resolver for Google Cloud TPUs.
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
# Connects to the given cluster.
tf.config.experimental_connect_to_cluster(tpu)
# Initialize the TPU devices.
tf.tpu.experimental.initialize_tpu_system(tpu)
# TPU distribution strategy implementation.
strategy = tf.distribute.experimental.TPUStrategy(tpu)

# Configuration
EPOCHS = 50
BATCH_SIZE = 16 * strategy.num_replicas_in_sync

INFO:tensorflow:Initializing the TPU system: grpc://10.108.225.122:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.108.225.122:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [6]:
def get_dataset():#اینجا دیگه واقعا گیر کردم اگر کافولد پایین رو بیارم بالا باید فور رو هم بیارم تا ایکس -تست و ایگرگ-تست رو بشناسه
    dataset = np.load('/content/drive/My Drive/data.row.battah/ASD2.npz') #Dataset ready in numpy array (removing background, resizing, and transforming into grayscale)
    X = dataset['X']
    y = dataset['y']

    return X,y

In [7]:
def GetModel():
    resnet50 = tf.keras.applications.ResNet50(weights='imagenet', include_top=True , input_tensor=None, input_shape=None, pooling=None)

    x = resnet50.layers[-2].output
    output = Dense(units=1, activation='sigmoid')(x)
    model = Model(inputs=resnet50.input, outputs=output)
    for layer in model.layers[175:]:
      layer.trainable = True
    for layer in model.layers[:175]:
      layer.trainable = False


    opt = SGD(lr=0.001,momentum=0.99,decay=0.01,nesterov=False)
    #opt = Adam(learning_rate=0.0001,beta_1=0.99,beta_2=0.99,epsilon=0.1,amsgrad=False,name="Adam",)

    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [8]:
#Hyperparameters
nfolds =3
nEpochs = 30
nBatch= 16
#inputDim = X.shape[1] # Count of features

In [9]:
def train_cross_validate(folds = 3):
    histories = []
    models = []

    X, y = get_dataset()

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 3)
    kfold = StratifiedKFold(n_splits=nfolds, shuffle=True, random_state=1).split(X, y) 
    

    for train, test in kfold:
        model = GetModel()
        history = model.fit(X[train], y[train], callbacks = [ early_stopping], validation_split= 0.2, epochs=nEpochs, batch_size=nBatch, verbose=2)
       
        models.append(model)
        histories.append(history)
        
    return histories, models

In [10]:
def predict(folds = 3):
   X, y = get_dataset()
   histories, models = train_cross_validate(folds = folds)
   

   scores=[]
   auc =[]


   for i in range(nfolds):
     result = models[i].evaluate(X[test], y[test], verbose=0)
     scores.append(result)
     y_pred = models[i].predict(X[test])
     AUC = roc_auc_score(y[test], y_pred.round())
     auc.append(AUC)  

   return scores, auc   

In [ ]:
scores, auc = predict(folds = 3)